In [29]:
# import packages
import pandas as pd
from gurobipy import *

# read excel sheets with pandas
data = pd.ExcelFile('SuperChipData.xlsx')
productionCapacityDf = pd.read_excel(data, 'Production Capacity')
salesRegionDemandDf = pd.read_excel(data, 'Sales Region Demand')
shippingCostsDf = pd.read_excel(data, 'Shipping Costs')
productionCostsDf = pd.read_excel(data, 'Production Costs')

# preparing for adding variables
# total number of sales region j
salesRegion = 23
# total number of computer chip z
computerChip = 30
# [sales region, computer chip, yearly demand(thousands)]
salesRegionDemand = salesRegionDemandDf.to_numpy()
# [facility name, facility yearly production capacity(thousands)]
productionCapacity = productionCapacityDf.to_numpy()
# [facility, computer chip, production cost per chip ($)]
productionCosts = productionCostsDf.to_numpy()
# [facility, computer chip, sales region, shipping cost per chip($)]
shippingCosts = shippingCostsDf.to_numpy()
print(shippingCosts)

# initialize model
m = Model()
# set objective function to be minimization
m.modelSense = GRB.MINIMIZE

# declare variables and add them to model
# facility set S
facilitySet = {"Alexandria", "Richmond", "Norfolk", "Roanoke", "Charlottesville"}

# Djz = z chip yearly demand in sales region j (thousands unit)
D = {}
for d in salesRegionDemand:
        # (j, z) = demand in j z
        D[d[0], d[1]] = d[2]

# Ci = yearly chip production capacity in facility i (thousands unit)
C = {}
for c in productionCapacity:
     # i = production capacity in i
     C[c[0]] = c[1]

# Piz = production cost per z chip in facility i ($)
P = {}
for p in productionCosts:
     # (i, z) = production cost in i, z
     P[p[0], p[1]] = p[2]

# Sijz = shipping cost per z chip from facility i to sales region j ($)
S = {}
for s in shippingCosts:
     # (i, j, z) = shipping cost per z from i to j
     S[s[0], s[2], s[1]] = s[3]

# Xijz = amount of z chips shipped from facility i to sales region j (unit)
X = {}
# iterate through facility set, sales region numbers and computer chip numbers
for i in range(len(facilitySet)):
    for j in range(1, salesRegion + 1):
        for z in range(1, computerChip + 1):
            X[i, j, z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "X" + 
                                  str(i) + str(j) + str(z))

# Yiz = amount of z chips produced in facility i (unit)
Y = {}
# iterate through facility set, computer chip numbers
for i in facilitySet:
    for z in range(1, computerChip + 1):
        Y[i, z] = m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "Y" + str(i) +
                           str(z))
        
# notify model the changes
m.update()

# set the objective function
m.setObjective(quicksum(S[i, j, z] * X[i, j, z] for i in facilitySet for j 
                        in range(1, salesRegion + 1) for z in 
                        range(1, computerChip + 1)) + 
               quicksum(P[i, z] * Y[i, z] for i in range(len(facilitySet)) for z in 
                        range(1, computerChip + 1)))

# first constraint ∑Yiz <= 1000Ci
facilityProductionConst = {}
for i in facilitySet:
     facilityProductionConst[i] = m.addConstr(quicksum(Y[i, z] for z in 
                                                          range(1, computerChip + 1)) <= 1000 * C[i])

# second constraint ∑Xijz = 1000Djz
shipDemandConst = {}
for j in range(1, salesRegion + 1):
     for z in range(1, computerChip + 1):
          shipDemandConst[j, z] = m.addConstr(quicksum(X[i, j, z] for i in facilitySet) == 1000 * D[j, z])

# third constraint Yiz >= ∑Xijz
productionShipConst = {}
for i in facilitySet:
     for z in range(1, computerChip + 1):
          productionShipConst[i, z] = m.addConstr(Y[i, z] >= quicksum(X[i, j, z] for j 
                                                                      in (1, salesRegion + 1)))

# for the fourth and fifth constraints, since Xijz and Yiz both have lower bound of
# 0 on initialization, not adding them here

# notify model the changes
m.update()

# trigger optimization
m.optimize()



[['Alexandria' 1 1 1.76]
 ['Alexandria' 2 1 1.74]
 ['Alexandria' 3 1 1.8]
 ...
 ['Charolottesville' 28 23 1.6]
 ['Charolottesville' 29 23 1.57]
 ['Charolottesville' 30 23 2.1]]


KeyError: ('Alexandria', 1, 1)